In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

from collections import Counter
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 3)
np.set_printoptions(linewidth=1000)

In [2]:
sei_supp_table_1 = pd.read_excel('../data/Sei/41588_2022_1102_MOESM3_ESM.xlsx', sheet_name='Supp Table 1', skiprows=1)
sei_supp_table_1

,Chromatin Profile,AUROC,AUPRC
0,HeLa_Epithelium_Cervix | BTAF1 | ID:1,0.983,0.033
1,H9_Embryonic_Stem_Cell_Embryo | EOMES | ID:1006,0.963,0.380
2,H9_Embryonic_Stem_Cell_Embryo | EP300 | ID:1007,0.987,0.516
3,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1010,0.954,0.207
4,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1011,0.962,0.135
...,...,...,...
21902,WERI-Rb-1 | CTCF | ENCODE,0.997,0.665
21903,WERI-Rb-1 | DNase | ENCODE,0.968,0.386
21904,WI-38 | CTCF | ENCODE,0.997,0.686
21905,WI-38 (4OHTAM_20nM_72hr) | DNase | ENCODE,0.963,0.385


In [3]:
sei_supp_table_1[['cell_name', 'assay', 'source', 'part']] = (
    sei_supp_table_1['Chromatin Profile']
    .str.split(r"\s*\|\s*", n=3, expand=True)
)
sei_supp_table_1['index'] = sei_supp_table_1.index
sei_supp_table_1

,Chromatin Profile,AUROC,AUPRC,cell_name,assay,source,part,index
0,HeLa_Epithelium_Cervix | BTAF1 | ID:1,0.983,0.033,HeLa_Epithelium_Cervix,BTAF1,ID:1,None,0
1,H9_Embryonic_Stem_Cell_Embryo | EOMES | ID:1006,0.963,0.380,H9_Embryonic_Stem_Cell_Embryo,EOMES,ID:1006,None,1
2,H9_Embryonic_Stem_Cell_Embryo | EP300 | ID:1007,0.987,0.516,H9_Embryonic_Stem_Cell_Embryo,EP300,ID:1007,None,2
3,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1010,0.954,0.207,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1010,None,3
4,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1011,0.962,0.135,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1011,None,4
...,...,...,...,...,...,...,...,...
21902,WERI-Rb-1 | CTCF | ENCODE,0.997,0.665,WERI-Rb-1,CTCF,ENCODE,None,21902
21903,WERI-Rb-1 | DNase | ENCODE,0.968,0.386,WERI-Rb-1,DNase,ENCODE,None,21903
21904,WI-38 | CTCF | ENCODE,0.997,0.686,WI-38,CTCF,ENCODE,None,21904
21905,WI-38 (4OHTAM_20nM_72hr) | DNase | ENCODE,0.963,0.385,WI-38 (4OHTAM_20nM_72hr),DNase,ENCODE,None,21905


In [4]:
sei_supp_table_2 = pd.read_excel('../data/Sei/41588_2022_1102_MOESM3_ESM.xlsx', sheet_name='Supp Table 2', skiprows=1)
sei_supp_table_2

,Original Cell Type Name,Standardized Celltype Name
0,1015c,1015c
1,10326,10326
2,1184_Stem_cell,1184_Stem_cell
3,133_Astrocyte,133_Astrocyte
4,143B_Osteosarcoma_cell,143B_Osteosarcoma_cell
...,...,...
1691,ZR-75-1_Breast,ZR-75-1
1692,ZR-75-1_Embryonic_Stem_Cell_Breast,ZR-75-1
1693,ZR-75-1_Epithelium_Mammary_Gland,ZR-75-1
1694,ZR-75-30,ZR-75-30_Epithelium_Mammary_Gland


In [5]:
cell_name_map_dict = sei_supp_table_2.set_index('Original Cell Type Name')['Standardized Celltype Name'].to_dict()
cell_name_map_dict

{'1015c': '1015c',
 '10326': '10326',
 '1184_Stem_cell': '1184_Stem_cell',
 '133_Astrocyte': '133_Astrocyte',
 '143B_Osteosarcoma_cell': '143B_Osteosarcoma_cell',
 '21NT_Epithelium_Breast': '21NT_Epithelium_Breast',
 '226LDM': '226LDM',
 '22RV1_Epithelium_Prostate': '22RV1',
 '293FT_Embryonic_Kidney': '293FT',
 '293T': '293T',
 '293T_Embryonic_Kidney': '293T',
 '293T-Rex': '293T-Rex',
 '3565_cells': '3565_cells',
 '451Lu_Melanoma_Cell': '451Lu_Melanoma_Cell',
 '4star': '4star',
 '501-Mel_melanoma': '501-Mel',
 '501-Mel_Melanoma_Cell': '501-Mel',
 '5637_Stem_cell': '5637_Stem_cell',
 '5637_Urinary_Bladder': '5637_Urinary_Bladder',
 '6540_cells_Epithelium': '6540_cells_Epithelium',
 '697_B-ALL': '697_B-ALL',
 '7250_Fibroblast': '7250_Fibroblast',
 '76NF2V_Breast': '76NF2V_Breast',
 '786-0_Kidney': '786-0_Kidney',
 '786-O_Epithelium_Kidney': '786-O_Epithelium_Kidney',
 '8988T': '8988T',
 '8988t_Epithelium_Pancreas': '8988T',
 '9.4_Neuron_Prefrontal': '9.4_Neuron_Prefrontal',
 '90-8TL_Schw

In [6]:
sei_supp_table_1['cell_type'] = sei_supp_table_1['cell_name'].map(cell_name_map_dict)
sei_supp_table_1

,Chromatin Profile,AUROC,AUPRC,cell_name,assay,source,part,index,cell_type
0,HeLa_Epithelium_Cervix | BTAF1 | ID:1,0.983,0.033,HeLa_Epithelium_Cervix,BTAF1,ID:1,None,0,HeLa_Cervical_Carcinoma
1,H9_Embryonic_Stem_Cell_Embryo | EOMES | ID:1006,0.963,0.380,H9_Embryonic_Stem_Cell_Embryo,EOMES,ID:1006,None,1,H9_Embryonic_Stem_Cell_Embryo
2,H9_Embryonic_Stem_Cell_Embryo | EP300 | ID:1007,0.987,0.516,H9_Embryonic_Stem_Cell_Embryo,EP300,ID:1007,None,2,H9_Embryonic_Stem_Cell_Embryo
3,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1010,0.954,0.207,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1010,None,3,H9_Embryonic_Stem_Cell_Embryo
4,H9_Embryonic_Stem_Cell_Embryo | FOXH1 | ID:1011,0.962,0.135,H9_Embryonic_Stem_Cell_Embryo,FOXH1,ID:1011,None,4,H9_Embryonic_Stem_Cell_Embryo
...,...,...,...,...,...,...,...,...,...
21902,WERI-Rb-1 | CTCF | ENCODE,0.997,0.665,WERI-Rb-1,CTCF,ENCODE,None,21902,WERI-Rb-1_Eye
21903,WERI-Rb-1 | DNase | ENCODE,0.968,0.386,WERI-Rb-1,DNase,ENCODE,None,21903,WERI-Rb-1_Eye
21904,WI-38 | CTCF | ENCODE,0.997,0.686,WI-38,CTCF,ENCODE,None,21904,WI-38_Fibroblast_Lung
21905,WI-38 (4OHTAM_20nM_72hr) | DNase | ENCODE,0.963,0.385,WI-38 (4OHTAM_20nM_72hr),DNase,ENCODE,None,21905,NaN


In [7]:
df_pivot = sei_supp_table_1.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)

df_pivot

assay                    ADNP AEBP2 AFF1 AFF4 AGO1 AGO2  AHR AHRR AP-2alpha AP-2gamma APOBEC3B   AR ARID1A ARID2 ARID3A ARID5B ARNT ARNT2 ARNTL ARRB1 ASCL1 ASCL2 ASH1L ASH2L ASXL1 ATAC-seq ATF1 ATF2 ATF3 ATF4 ATF5 ATF7IP ATOH1 ATRX AUTS2 BACH1 BACH2 BAF155 BAF170 BAHD1 BANF1 BANP BARX1 BARX2 BATF BATF3 BCAT1 BCL11A BCL11B BCL3 BCL6 BCLAF1 BCOR BDP1 BHLHE40 BIRA BMI1 BPTF BRAF BRCA1 BRD1 BRD2    BRD3    BRD4 BRD7 BRD9 BRDU BRF1 BRF2 BRPF3 BTAF1 Bach1 Brg1 C11orf30 C17orf49 C17orf96 CAMTA2 CASP8AP2 CBFA2T2 CBFB CBX1 CBX2 CBX3 CBX4 CBX5 CBX6 CBX7 CBX8 CCDC101 CCND2 CCNT2 CD59 CD74 CDC5L CDCA2 CDK12 CDK2 CDK6 CDK7 CDK8 CDK9 CDX2 CEBPA CEBPB CEBPD CEBPG CEBPZ CENPA CENPC CENPT CHAMP1 CHAT CHD1 CHD2 CHD4 CHD5 CHD7 CHD8 CHRM2 CIITA CLOCK CNOT3 COPS2 COREST CPSF3L CREB1 CREB3 CREB3L4 CREBBP CREM CRY1 CSDE1 CSNK2A1 CTBP1 CTBP2 CTCF CTCFL CTNNB1 CUX1 CtBP2 DACOR1 DAXX DCP1A DDX20 DDX21 DDX5 DEAF1  DEK DIDO1 DLX1 DLX2 DMC1 DNMT1 DNMT3A DNMT3B    DNase DNase.all.peaks DNase.fdr0.01.hot  \
cell_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1015c                     NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN      NaN  NaN    NaN   NaN    NaN    NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN  NaN  NaN  NaN  NaN  NaN    NaN   NaN  NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN    NaN    NaN  NaN  NaN    NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN     NaN  [8996]  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  NaN      NaN      NaN      NaN    NaN      NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN     NaN    NaN  NaN  NaN   NaN     NaN   NaN   NaN  NaN   NaN    NaN  NaN   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN   NaN    NaN    NaN      NaN             NaN               NaN   
10326                     NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN      NaN  NaN    NaN   NaN    NaN    NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN  NaN  NaN  NaN  NaN  NaN    NaN   NaN  NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN    NaN    NaN  NaN  NaN    NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN  [8997]     NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  NaN      NaN      NaN      NaN    NaN      NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN     NaN    NaN  NaN  NaN   NaN     NaN   NaN   NaN  NaN   NaN    NaN  NaN   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN   NaN    NaN    NaN      NaN             NaN               NaN   
1184_Stem_cell            NaN   

In [8]:
df_avail = df_pivot[['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']]
df_avail = df_avail[(df_avail.isna().sum(axis=1) == 0)]
df_avail

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
A549,"[3147, 4493, 4494, 19920]","[1664, 1668, 2466, 2470, 2613, 2621, 2623, 471...","[2478, 2487, 4707, 4715, 10889, 10890, 10891, ...","[4704, 4706, 5075, 5076, 5081, 5083, 5563, 557..."
AG04450_Fibroblast_Fetal_Lung,"[4497, 19983]",[4724],[4725],"[5088, 5089, 19982]"
Astrocyte_Brain,"[3235, 4651, 4652]",[4991],[4993],[4986]
BE2-C,"[4506, 4507, 20000]","[4730, 9964, 18290]","[9233, 9238, 10592, 10593, 10632]","[5098, 5099, 19999]"
BJ_Fibroblast_Foreskin,"[4508, 4509, 7138, 10659, 10660, 10684, 10685,...","[4733, 14000, 14003, 14005, 14279]","[7961, 10582, 17690]","[5100, 5101, 20001]"
B_Lymphocyte_Blood,"[635, 3146, 4516, 4517]","[919, 4740, 4742, 7037]","[2052, 4281, 4282, 4741, 7637, 7638]",[4737]
Brain,"[10678, 10679, 10680, 16416]","[2324, 2325, 2327, 2443, 9384, 9388, 9391, 939...","[2205, 9371, 9372, 9373, 9374, 9375, 9376, 937...","[5104, 5519]"
Caco-2_Epithelium_Colon,"[4514, 4515, 16625, 20080]",[4735],[17327],"[5102, 5103, 20079]"
Embryonic_Stem_Cell_Embryo,[3016],"[1705, 2210, 4098, 4102, 4106, 6849, 6850, 6983]","[2204, 2212, 4099, 4103, 4107, 9947, 9948, 995...","[9949, 9950, 9957, 9958]"


In [9]:
print(len(df_avail))

61


In [17]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
cell_types_standard_names = ['HepG2_Hepatocellular_Carcinoma', 'K562_Leukemia_Cell', 'SK-N-SH_Neuroblastoma_cell_Brain', 'A549', 'HCT-116_Colorectal_cancer_cell_line', 'SK-N-SH_RA_Neuron_Brain']

In [18]:
df_Sei_Siraj = df_pivot.loc[cell_types_standard_names][assays]
df_Sei_Siraj.map(lambda x: len(x) if isinstance(x, list) else 0)

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
HepG2_Hepatocellular_Carcinoma,9,8,9,17
K562_Leukemia_Cell,58,91,51,29
SK-N-SH_Neuroblastoma_cell_Brain,0,3,3,2
A549,4,45,53,10
HCT-116_Colorectal_cancer_cell_line,6,45,49,14
SK-N-SH_RA_Neuron_Brain,4,1,0,5


In [19]:
pred_array = np.load('../pretrained_models/Sei/Sei_Siraj_pred_562654.npy')
print(pred_array.shape)

(562654, 21907)


In [20]:
MPRA_df = pd.read_csv('../data/SirajMPRA/SirajMPRA_562654.csv')
print(MPRA_df.shape)

(562654, 15)


In [24]:
# average every channel

pred_feature_matrix = np.full(shape=(len(pred_array), len(cell_types), len(assays)), fill_value=np.nan)

pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

np.full(shape=(len(cell_types), len(assays)), fill_value=np.nan)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        cell_name = cell_types_standard_names[i]
        if cell_type == 'SK-N-SH' and assay == 'DNase':
            # mannual select SK-N-SH_RA
            cell_name = 'SK-N-SH_RA_Neuron_Brain'

        indice = df_Sei_Siraj.loc[cell_name, assay]
        if indice is not None:
            predicted_feature = logit(pred_array[:, indice]).mean(1)
            pred_feature_matrix[:, i, j] = predicted_feature
            true_label = MPRA_df[cell_type]
            r, _ = pearson(predicted_feature, true_label)
            pearson_df.loc[cell_type, assay] = r
            
pearson_df

         DNase  H3K4me3  H3K27ac  CTCF
HepG2      NaN      NaN      NaN   NaN
K562       NaN      NaN      NaN   NaN
SK-N-SH    NaN      NaN      NaN   NaN
A549       NaN      NaN      NaN   NaN
HCT116     NaN      NaN      NaN   NaN


In [26]:
pearson_df

,DNase,H3K4me3,H3K27ac,CTCF
HepG2,0.443,0.469,0.427,0.350
K562,0.499,0.451,0.444,0.364
SK-N-SH,0.368,0.486,0.470,0.298
A549,0.348,0.362,0.363,0.247
HCT116,0.424,0.443,0.452,0.329


In [27]:
np.save('Sei_Siraj_pred_feature_matrix.npy', pred_feature_matrix)

In [37]:
###### if model is enformer

model_info_df = pd.read_excel('../data/Enformer/41592_2021_1252_MOESM3_ESM.xlsx', sheet_name='Supplementary Table 2')
model_info_df

,index,genome,identifier,file,clip,scale,sum_stat,description,assay_type,target,assay_subtype,DNASE: cell-type agnostic,CAGE: cell-type agnostic,CAGE: IRF6,CAGE: TERT-GBM,CAGE: ZFAND3,"CAGE: HNF4A,MSMB,TERT-HEK293T,MYCrs6983267","CAGE: GP1BB,HBB,HBG1,PKLR","CAGE: F9,LDLR,SORT1",DNASE: IRF4,DNASE: IRF6,DNASE: ZFAND3,"DNASE: HNF4A,MSMB,TERT-HEK293T,MYCrs6983267","DNASE: GP1BB,HBB,HBG1,PKLR","DNASE: F9,LDLR,SORT1"
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,DNase/cerebellum male adult (27 years) and mal...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,DNase/frontal cortex male adult (27 years) and...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,DNase/chorion,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,DNase/Ishikawa treated with 0.02% dimethyl sul...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,DNase/GM03348,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5308,5308,0,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,CAGE/epithelioid sarcoma cell line:HS-ES-2R,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5309,5309,0,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,CAGE/squamous cell lung carcinoma cell line:RE...,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5310,5310,0,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,CAGE/gastric cancer cell line:GSS,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5311,5311,0,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,CAGE/carcinoid cell line:NCI-H727,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [51]:
model_info_df = model_info_df[['index', 'target']].copy()
model_info_df[['assay', 'cell_type']] = model_info_df['target'].str.split('/', n=1, expand=True)
model_info_df['assay'] = model_info_df['assay'].str.replace('ChIP-Histone:', '').str.replace('ChIP-TF:', '')
model_info_df

,index,target,assay,cell_type
0,0,DNase/cerebellum male adult (27 years) and mal...,DNase,cerebellum male adult (27 years) and male adul...
1,1,DNase/frontal cortex male adult (27 years) and...,DNase,frontal cortex male adult (27 years) and male ...
2,2,DNase/chorion,DNase,chorion
3,3,DNase/Ishikawa treated with 0.02% dimethyl sul...,DNase,Ishikawa treated with 0.02% dimethyl sulfoxide...
4,4,DNase/GM03348,DNase,GM03348
...,...,...,...,...
5308,5308,CAGE/epithelioid sarcoma cell line:HS-ES-2R,CAGE,epithelioid sarcoma cell line:HS-ES-2R
5309,5309,CAGE/squamous cell lung carcinoma cell line:RE...,CAGE,squamous cell lung carcinoma cell line:RERF-LC-AI
5310,5310,CAGE/gastric cancer cell line:GSS,CAGE,gastric cancer cell line:GSS
5311,5311,CAGE/carcinoid cell line:NCI-H727,CAGE,carcinoid cell line:NCI-H727


In [52]:
df_pivot = model_info_df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)
df_pivot

assay,.,3xFLAG-AHR,3xFLAG-ARID4B,3xFLAG-ATF1,3xFLAG-ATF4,3xFLAG-BCL6,3xFLAG-CEBPA,3xFLAG-CEBPG,3xFLAG-CREB1,3xFLAG-DMAP1,3xFLAG-DNMT3B,3xFLAG-DRAP1,3xFLAG-ELF3,3xFLAG-ERF,3xFLAG-ETV5,3xFLAG-FOXA3,3xFLAG-FOXP1,3xFLAG-GABPA,3xFLAG-GABPB1,3xFLAG-GATAD1,3xFLAG-GATAD2A,3xFLAG-GMEB2,3xFLAG-HBP1,3xFLAG-HHEX,3xFLAG-HLF,3xFLAG-HMG20A,3xFLAG-HMG20B,3xFLAG-HOMEZ,3xFLAG-IKZF5,3xFLAG-IRF2,3xFLAG-KAT7,3xFLAG-KAT8,3xFLAG-KDM1A,3xFLAG-KDM3A,3xFLAG-KDM6A,3xFLAG-KLF11,3xFLAG-KLF16,3xFLAG-KLF6,3xFLAG-KLF9,3xFLAG-KMT2B,3xFLAG-MBD1,3xFLAG-MIER2,3xFLAG-MIER3,3xFLAG-MIXL1,3xFLAG-MLX,3xFLAG-MXD3,3xFLAG-MXD4,3xFLAG-MYRF,3xFLAG-NFIA,3xFLAG-NFIL3,3xFLAG-NFKBIZ,3xFLAG-NFYC,3xFLAG-NR2F1,3xFLAG-NR2F6,3xFLAG-PAF1,3xFLAG-PBX2,3xFLAG-PHF5A,3xFLAG-PPARG,3xFLAG-RAD21,3xFLAG-RARA,3xFLAG-RBPJ,3xFLAG-RCOR2,3xFLAG-RERE,3xFLAG-RFX3,3xFLAG-RFXANK,3xFLAG-RXRB,3xFLAG-SAP130,3xFLAG-SLC30A9,3xFLAG-SMAD4,3xFLAG-SOX13,3xFLAG-SOX5,3xFLAG-SP5,3xFLAG-SSRP1,3xFLAG-TEAD1,3xFLAG-TEAD3,3xFLAG-TFE3,3xFLAG-TGIF2,3xFLAG-THAP11,3xFLAG-THRB,3xFLAG-ZBED5,3xFLAG-ZBTB25,3xFLAG-ZBTB26,3xFLAG-ZFP1,3xFLAG-ZFP64,3xFLAG-ZGPAT,3xFLAG-ZKSCAN8,3xFLAG-ZNF146,3xFLAG-ZNF205,3xFLAG-ZNF3,3xFLAG-ZNF48,3xFLAG-ZNF511,3xFLAG-ZNF580,3xFLAG-ZNF614,3xFLAG-ZNF639,3xFLAG-ZNF644,3xFLAG-ZNF652,3xFLAG-ZNF7,3xFLAG-ZNF792,3xFLAG-ZSCAN9,AFF1,AGO1,AGO2,ARHGAP35,ARID1B,ARID2,ARID3A,ARNT,ASH1L,ASH2L,ATAC,ATF2,ATF3,ATF4,ATF7,ATM,BACH1,BATF,BCL11A,BCL3,BCLAF1,BCOR,BHLHE40,BMAL1,BMI1,BRCA1,BRD4,BRD9,C11orf30,CAGE,CBFA2T2,CBFA2T3,CBFB,CBX1,CBX2,CBX3,CBX5,CBX8,CC2D1A,CCAR2,CDC5L,CEBPB,CEBPZ,CEBPb,CHAMP1,CHD1,CHD2,CHD4,CHD7,CLOCK,COPS2,CREB1,CREB3L1,CREBBP,CREM,CSDE1,CTBP1,CTBP2,CTCF,CUX1,DACH1,DDX20,DEAF1,DEK,DNMT1,DNase,DPF2,E2F1,E2F4,E2F6,E2F7,E2F8,E4F1,EBF1,EED,EGR1,EHMT2,ELF1,ELF4,ELK1,EP300,EP400,ESR1,ESRRA,ETS1,ETV4,ETV6,EWSR1,EZH2,EZH2phosphoT487,FIP1L1,FOS,FOSL1,FOSL2,FOXA1,FOXA2,FOXK2,FOXM1,FOXP1,FOXP2,FUS,GABPA,GABPB1,GATA1,GATA2,GATA3,GATA4,GATAD2A,GATAD2B,GMEB1,GR,GTF2F1,H2AFZ,H2AK5ac,H2AK9ac,H2BK120ac,H2BK12ac,H2BK15ac,H2BK20ac,H2BK5ac,H3F3A,H3K14ac,H3K18ac,H3K23ac,H3K23me2,H3K27Ac,H3K27ac,H3K27me3,H3K36me3,H3K4ac,H3K4me1,H3K4me2,H3K4me3,H3K56ac,H3K79me1,H3K79me2,H3K9ac,H3K9me1,H3K9me2,H3K9me3,H3T11ph,H4K12ac,H4K20me1,H4K5ac,H4K8ac,H4K91ac,HCFC1,HDAC1,HDAC2,HDAC3,HDAC6,HDGF,HES1,HLTF,HMBOX1,HNF1A,HNF4A,HNF4G,HNRNPH1,HNRNPK,HNRNPL,HNRNPLL,HNRNPUL1,HSF1,IKZF1,IKZF2,ILF3,IRF1,IRF2,IRF3,IRF4,IRF5,JUN,JUNB,JUND,KAT2A,KAT2B,KAT8,KDM1A,KDM4A,KDM4B,KDM5A,KDM5B,KHSRP,KLF16,KLF5,L3MBTL2,LARP7,LCORL,LEF1,MAFF,MAFK,MAX,MAZ,MBD2,MCM2,MCM3,MCM5,MCM7,MEF2A,MEF2B,MEF2C,MEIS2,MGA,MIER1,MITF,MLLT1,MNT,MTA1,MTA2,MTA3,MXI1,MYB,MYBL2,MYC,MYNN,NANOG,NBN,NCOA1,NCOA2,NCOA3,NCOA4,NCOA6,NCOR1,NEUROD1,NFATC1,NFATC3,NFE2,NFE2L2,NFIB,NFIC,NFRKB,NFXL1,NFYA,NFYB,NKRF,NONO,NR0B1,NR2C1,NR2C2,NR2F1,NR2F2,NR2F6,NR3C1,NRF1,NUFIP1,PAX5,PAX8,PBX3,PCBP1,PCBP2,PHB2,PHF20,PHF21A,PHF8,PKNOX1,PLRG1,PML,POLR2A,POLR2AphosphoS2,POLR2AphosphoS5,POLR2B,POLR2G,POU2F2,POU5F1,PPP1R10,PRDM10,PRPF4,PTBP1,PYGO2,RAD21,RAD51,RB1,RBBP5,RBFOX2,RBM14,RBM15,RBM17,RBM22,RBM25,RBM34,RBM39,RCOR1,RELB,REST,RFX1,RFX5,RLF,RNAPII,RNF2,RUNX1,RUNX3,RXRA,SAFB,SAFB2,SAP30,SETDB1,SFPQ,SIN3A,SIN3B,SIRT6,SIX4,SIX5,SKI,SKIL,SMAD1,SMAD2,SMAD5,SMARCA4,SMARCA5,SMARCB1,SMARCC2,SMARCE1,SMC3,SNIP1,SNRNP70,SOX13,SOX6,SP1,SPI1,SREBF1,SREBF2,SRF,SRSF1,SRSF3,SRSF4,SRSF7,SRSF9,STAT1,STAT2,STAT3,STAT5A,SUPT20H,SUPT5H,SUZ12,SYNCRIP,TAF1,TAF15,TAF7,TAF9B,TAL1,TARDBP,TBL1XR1,TBP,TBX21,TBX3,TCF12,TCF7,TCF7L2,TEAD4,TFAP4,THAP1,THRA,THRAP3,TOE1,TRIM22,TRIM24,TRIM25,TRIM28,TRIP13,U2AF1,U2AF2,UBTF,USF1,USF2,WHSC1,WRNIP1,XRCC3,XRCC5,YBX1,YBX3,YY1,ZBED1,ZBTB1,ZBTB11,ZBTB2,ZBTB33,ZBTB40,ZBTB5,ZBTB7A,ZBTB7B,ZBTB8A,ZC3H11A,ZC3H8,ZEB1,ZEB2,ZFP36,ZFP91,ZFX,ZHX1,ZHX2,ZKSCAN1,ZMIZ1,ZMYM3,ZNF143,ZNF184,ZNF207,ZNF217,ZNF24,ZNF263,ZNF274,ZNF280A,ZNF282,ZNF316,ZNF318,ZNF384,ZNF407,ZNF444,ZNF507,ZNF512B,ZNF574,ZNF579,ZNF592,ZNF622,ZNF639,ZNF687,ZNF8,ZNF830,ZSCAN29,ZZZ3,abcam,active,eGFP-ADNP,eGFP-AEBP2,eGFP-ATF1,eGFP-ATF2,eGFP-ATF3,eGFP-BACH1,eGFP-BCL11A,eGFP-BCL11B,eGFP-BCL6B,eGFP-CEBPB,eGFP-CEBPG,eGFP-CREB3,eGFP-CT

In [53]:
df_pivot.columns

Index(['.', '3xFLAG-AHR', '3xFLAG-ARID4B', '3xFLAG-ATF1', '3xFLAG-ATF4', '3xFLAG-BCL6', '3xFLAG-CEBPA', '3xFLAG-CEBPG', '3xFLAG-CREB1', '3xFLAG-DMAP1',
       ...
       'eGFP-ZSCAN21', 'eGFP-ZSCAN23', 'eGFP-ZSCAN26', 'eGFP-ZSCAN30', 'eGFP-ZSCAN4', 'eGFP-ZSCAN5A', 'eGFP-ZSCAN5C', 'eGFP-ZXDB', 'hBMAL1', 'hHIF1A'], dtype='object', name='assay', length=791)

In [54]:
df_avail = df_pivot[['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']]
df_avail = df_avail[(df_avail.isna().sum(axis=1) == 0)]
df_avail

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
A673,[340],[2783],[3657],[3336]
AG04450,[49],[1109],[1107],[1108]
CD14-positive monocyte female,"[41, 131]","[821, 1215]",[817],[836]
GM12878,"[12, 69]","[686, 1143, 1620]",[688],"[687, 1265]"
GM23338 male adult (53 years) originated from GM23248,[166],[3463],[3702],"[4324, 4474]"
H1-hESC,"[19, 70, 565]","[1509, 3972]","[732, 4164]","[712, 989, 1074]"
HCT116,[88],"[1173, 2467]","[1379, 3485]","[1055, 1172, 2203]"
HeLa-S3,"[25, 654]","[745, 2493]",[743],[742]
IMR-90,[405],[1719],[1452],[1322]


In [55]:
len(df_avail)

41

In [56]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']

In [57]:
df_Sei_Siraj = df_pivot.loc[cell_types][assays]
df_Sei_Siraj.map(lambda x: len(x) if isinstance(x, list) else 0)

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
HepG2,3,3,0,1
K562,4,4,0,4
SK-N-SH,0,2,2,2
A549,1,1,0,2
HCT116,1,2,2,3


In [10]:
model_info_df = pd.read_csv('../pretrained_models/Sei/Sei_info.csv')
MPRA_df = pd.read_csv('../data/SirajMPRA/SirajMPRA_562654.csv')

In [11]:
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']

# # 定义细胞所有可能的名字
cell_names_dict = {
    'HepG2': [
        r"HepG2",
        r"Hep-G2",
        ],
    'K562': [
        r"K562",
        r"K-562",
    ],
    'SK-N-SH': [
        r"SK-N-SH",
        r"SKNSH",
        r"SKN-SH",
        r"SK-NSH",
        r"neuroblastoma",
    ],
    'A549': [
        r"A549",
        r"A-549",
    ],
    'HCT116': [
        r"HCT116",
        r"HCT-116",
    ],
}


for cell_type in cell_names_dict:
    cell_names = cell_names_dict[cell_type]
    pattern = re.compile("|".join(cell_names), re.IGNORECASE) # 不区分大小写
    matched_indice = [i for i in range(len(model_info_df))
                      if (pattern.search(model_info_df['cell_type'][i])) and (model_info_df['assay'][i] in assays)]

    counter = Counter(model_info_df.iloc[matched_indice]['cell_type'])
    for a in counter.most_common():
        print(a)
    print()

('HepG2_Epithelium_Liver', 42)
('HepG2_Hepatocellular_Carcinoma', 7)
('HepG2', 5)

('K562_Erythroblast_Bone_Marrow', 284)
('K562', 13)
('K562_Leukemia', 8)
('K562_Leukemia_Bone', 2)
('K562_human_erythroleukemic_cell_line', 1)

('Neuroblastoma_cell', 16)
('SHEP-21N_Neuroblastoma_cell_Bone_Marrow', 12)
('SK-N-SH_RA_Neuron_Brain', 9)
('SK-N-SH_Neuroblastoma_cell_Brain', 8)
('IMR-5_75_Neuroblastoma_cell', 4)
('SH-SY5Y_Neuroblastoma_cell', 4)
('Neuroblastoma_patient_cells', 3)
('SK-N-SH_RA', 3)
('BE2-C_Neuroblastoma_cell', 2)
('LAN-1_Neuroblastoma_cell', 2)
('CLB-Ga_Neuroblastoma_cell', 2)
('NB-1643_Neuroblastoma_cell', 1)
('SK-N-BE(2)c_Neuroblastoma_cell', 1)
('SJNB-8_Neuroblastoma_cell', 1)
('SK-N-DZ_Neuroblastoma_cell', 1)
('IMR-32_Neuroblastoma_cell', 1)
('SK-N-AS_Neuroblastoma_cell', 1)
('SJNB-6_Neuroblastoma_cell', 1)
('EBC-1_Neuroblastoma_cell', 1)
('KELLY_Neuroblastoma_cell', 1)
('CHP-212_Neuroblastoma_cell', 1)
('GI-CA-N_Neuroblastoma_cell', 1)
('SH-EP_Neuroblastoma_cell', 1)
('SJN

In [12]:
# # 定义细胞所有可能的名字
cell_names_dict = {
    'HepG2'  : ["HepG2_Epithelium_Liver", "HepG2_Hepatocellular_Carcinoma", "HepG2"],
    'K562'   : ["K562_Erythroblast_Bone_Marrow", "K562"],
    'SK-N-SH': ["SK-N-SH_RA_Neuron_Brain", "SK-N-SH_Neuroblastoma_cell_Brain", "SK-N-SH_RA"],
    #'A549'   : ["A549_Epithelium_Lung", "A549"],
    'A549'   : ["A549_Epithelium_Lung"],
    'HCT116' : ["HCT-116_Colon", "HCT-116_HCT116", "HCT-116"],
}


cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
index_df = pd.DataFrame(index=cell_types, columns=assays)

for cell_type in cell_types:
    for assay in assays:
        indice = []
        for cell_name in cell_names_dict[cell_type]:
            mask = (model_info_df['cell_type'].str.upper() == cell_name.upper()) & (model_info_df['assay'] == assay) # 不区分大小写
            index = model_info_df[mask].index.tolist()
            indice.extend(index)
        index_df.loc[cell_type, assay] = indice

index_df_len = index_df.map(len)
index_df_len

,DNase,H3K4me1,H3K4me3,H3K9me3,H3K27me3,H3K27ac,H3K36me3,CTCF
HepG2,9,4,8,2,2,9,3,17
K562,58,10,91,21,34,51,5,27
SK-N-SH,4,0,4,0,1,3,1,7
A549,3,37,45,27,3,53,1,8
HCT116,7,18,39,1,1,49,4,14


In [13]:
predicted_array = np.load('../pretrained_models/Sei/Sei_Siraj_pred_562654.npy')

In [14]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

feature_matrix = np.full(shape=(len(predicted_array), len(cell_types), len(assays)), fill_value=np.nan)
print(feature_matrix.shape)

(562654, 5, 4)


In [17]:
# average every channel

pearson_array = np.zeros((len(cell_types), len(assays)))
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = index_df.iloc[i, j]
        if indice:
            predicted_feature = logit(predicted_array[:, indice]).mean(1)
            true_label = MPRA_df[cell_type]
            r, _ = pearson(predicted_feature, true_label)
            feature_matrix[:, i, j] = predicted_feature
            pearson_array[i, j] = r

print(pearson_array)
# print(feature_matrix.mean(-1))

[[ 0.443  0.351  0.469 -0.011]
 [ 0.499  0.388  0.451  0.285]
 [ 0.368  0.     0.462  0.   ]
 [ 0.345  0.304  0.362  0.237]
 [ 0.426  0.435  0.446 -0.057]]


In [11]:
# # if use less cell name
# cell_names_dict = {
#     'HepG2'  : ["HepG2_Epithelium_Liver"],
#     'K562'   : ["K562_Erythroblast_Bone_Marrow"],
#     'SK-N-SH': ["SK-N-SH_RA_Neuron_Brain", "SK-N-SH_Neuroblastoma_cell_Brain", "SK-N-SH_RA"],
#     'A549'   : ["A549_Epithelium_Lung"],
#     'HCT116' : ["HCT-116_Colon", "HCT-116_HCT116"],
# }


# cell_types = ['K562', 'HepG2', 'SK-N-SH', 'A549', 'HCT116']
# assays = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27me3', 'H3K27ac', 'H3K36me3', 'CTCF']
# index_list = []
# index_df = pd.DataFrame(index=cell_types, columns=assays)

# for cell_type in cell_types:
#     for assay in assays:
#         indice = []
#         for cell_name in cell_names_dict[cell_type]:
#             mask = (model_info_df['cell_type'].str.upper() == cell_name.upper()) & (model_info_df['assay'] == assay) # 不区分大小写
#             index = model_info_df[mask].index.tolist()
#             indice.extend(index)

#         index_df.loc[cell_type, assay] = indice
#         index_list.extend(indice)

# index_df_len = index_df.map(len)
# print(index_df_len)


# pearson_array = np.zeros((len(cell_types), len(assays)))
# for i, cell_type in enumerate(cell_types):
#     for j, assay in enumerate(assays):
#         indice = index_df.iloc[i, j]
#         if indice:
#             predicted_feature = logit(predicted_array[:, indice]).mean(1)
#             true_label = MPRA_df[cell_type]
#             r = pearson(predicted_feature, true_label)
#             feature_matrix[:, i, j] = predicted_feature
#             pearson_array[i, j] = r

# print(pearson_array)